# Publish Model to DLHub
Publish the FCHL model to DLHub

In [1]:
from dlhub_sdk.models.servables.python import PythonStaticMethodModel
from dlhub_sdk.utils.types import compose_argument_block
from dlhub_sdk.client import DLHubClient
from dlhub_app import evaluate_molecules
from time import sleep
import numpy as np

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


## Start the Servable Description
Load in the function information, and build a basic description

In [2]:
model = PythonStaticMethodModel.from_function_pointer(evaluate_molecules)

Give the model a title and name

In [3]:
model.set_name('g4mp2_delta_fchl').set_title('FCHL Model to Predict G4MP2 Activation Energy from B3LYP Energy and Coordinates')

Describe the function inputs

In [4]:
model.set_inputs('tuple', 'Structures and B3LYP energies of moelcule sto be evaluated',
                 element_types=[
                     compose_argument_block('list', 'Structures of molecules in XYZ format', item_type='string'),
                     compose_argument_block('list', 'B3LYP total energies of Energies of molecules in Ha', item_type='float')
                 ])

In [5]:
model.set_outputs('list', 'Estimate of G4MP2 atomization energy', item_type='float')

In [6]:
model.set_unpack_inputs(True)

## Add Datacite Information
Define who made the model, who should be credited, etc.

In [7]:
model.set_authors(["Ward, Logan", "Blaiszik, Ben", "Foster, Ian", "Assary, Rajeev", "Narayanan, Badri", "Curtis, Larry"],
                 [["Argonne National Laboratory"]]*6)

In [8]:
model.add_related_identifier("10.5281/zenodo.817332", "DOI", "Requires")

## Add Relevant Files
We need the model and some utility functions

In [9]:
model.add_file('dlhub_app.py')

In [10]:
model.add_file('model.pkl.gz')

In [11]:
model.add_directory('jcesr_ml', recursive=True, include='*.py')

## Describe Computational Environment
We described our environment using the `repo2docker` configuration files

In [12]:
model.parse_repo2docker_configuration()

In [13]:
model.list_files()

['dlhub_app.py',
 'model.pkl.gz',
 'jcesr_ml/atom_environment.py',
 'jcesr_ml/benchmark.py',
 'jcesr_ml/coordinates.py',
 'jcesr_ml/qml.py',
 'jcesr_ml/schnetpack.py',
 'jcesr_ml/style.py',
 'jcesr_ml/utils.py',
 'jcesr_ml/__init__.py',
 '/mnt/c/Users/ward6/Documents/UC/JCESR/g4mp2-atomization-energy/all-train-data/fchl-delta/environment.yml',
 '/mnt/c/Users/ward6/Documents/UC/JCESR/g4mp2-atomization-energy/all-train-data/fchl-delta/apt.txt']

## Publish it to DLHub
Make a client and send the model in

In [14]:
client = DLHubClient(http_timeout=-1)

In [15]:
task_id = client.publish_servable(model)

In [16]:
print(f'Submission ID: {task_id}')

Submission ID: 0ca88eb3-b314-42f0-a499-6118a41ba2fa


In [17]:
while client.get_task_status(task_id)['status'] != 'SUCCEEDED':
    sleep(30)

In [18]:
client.get_task_status(task_id)

{'status': 'SUCCEEDED',
 'output': '{"dlhub": {"files": {"other": ["dlhub_app.py", "model.pkl.gz", "jcesr_ml/atom_environment.py", "jcesr_ml/benchmark.py", "jcesr_ml/coordinates.py", "jcesr_ml/qml.py", "jcesr_ml/schnetpack.py", "jcesr_ml/style.py", "jcesr_ml/utils.py", "jcesr_ml/__init__.py", "environment.yml", "apt.txt"]}, "transfer_method": {"path": "/mnt/tmp/servable.zip", "POST": "file"}, "user_id": 3, "name": "g4mp2_delta_fchl", "ecr_arn": "arn:aws:ecr:us-east-1:039706667969:repository/cbd34e70-52ee-4ba8-b201-f209a15d7ffd", "visible_to": ["public"], "funcx_id": "757264d9-30c2-434e-b903-c48df0fa335c", "owner": "loganw_globusid", "ecr_uri": "039706667969.dkr.ecr.us-east-1.amazonaws.com/cbd34e70-52ee-4ba8-b201-f209a15d7ffd", "version": "0.8.3", "build_location": "/mnt/dlhub_ingest/cbd34e70-52ee-4ba8-b201-f209a15d7ffd-1569434890", "shorthand_name": "loganw_globusid/g4mp2_delta_fchl", "domains": [], "publication_date": 1569434890028, "identifier": "globus:GsgMGXDsu5Fc", "type": "servab

## Make Sure Model Works
See how the predictions from DLHub compare to running the model locally

In [19]:
model.name

'g4mp2_delta_fchl'

In [29]:
mols = ['3\nH2 O1\nO -0.034360 0.977540 0.007602\nH 0.064766 0.020572 0.001535\nH 0.871790 1.300792 0.000693']
b3lyp = [-76.404702]

In [30]:
%%time
dlhub_result = client.run(client.get_username() + '/' + model.name, (mols, b3lyp))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 27.5 s


In [31]:
%%time
local_result = evaluate_molecules(mols, b3lyp)

CPU times: user 47.3 s, sys: 4.73 s, total: 52.1 s
Wall time: 28.9 s


In [32]:
assert np.isclose(local_result, dlhub_result).all()